In [1]:
import os

In [2]:
%pwd

'/Users/mitpatel/Downloads/Code/GoJo-Rika/MLOPS/data-science-project-with-deployment/research'

In [3]:
os.chdir("../")
%pwd

'/Users/mitpatel/Downloads/Code/GoJo-Rika/MLOPS/data-science-project-with-deployment'

In [4]:
import pandas as pd

data = pd.read_csv("artifacts/data_ingestion/winequality-red.csv")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [6]:
data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [7]:
data.shape

(1599, 12)

In [8]:
from dataclasses import dataclass
from pathlib import Path


@dataclass
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [9]:
from src.data_science.constants import *
from src.data_science.utils.common import read_yaml, create_directories

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir=config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config

In [11]:
import os
from src.data_science import logger

In [12]:
class DataValiadtion:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self)-> bool:
        try:

            data = pd.read_csv(self.config.unzip_data_dir)
            # Extract actual column names and dtypes from the DataFrame
            all_cols = list(data.columns)
            all_dtypes = [col_dtype.name for col_dtype in data.dtypes.values]

            # Extract schema column names and expected dtypes
            all_schema_name = self.config.all_schema.keys()
            all_schema_dtype = list(self.config.all_schema.values())

            # Check if all columns in the data match the schema columns (order doesn't matter)
            all_columns_match = set(all_cols) == set(all_schema_name)

            # Check if all datatypes match between schema and data (order matters)
            all_dtypes_match = (
                all(a == b for a, b in zip(all_schema_dtype, all_dtypes)) and
                len(all_schema_dtype) == len(all_dtypes)
            )

            # Final validation: both columns and dtypes must match
            validation_status = all_columns_match and all_dtypes_match

            # Write validation results to a status file
            with open(self.config.STATUS_FILE, 'w') as f:
                f.write(f"Validation status: {validation_status}\n")
                # If columns do not match, write details about missing/extra columns
                if not all_columns_match:
                    f.write(f"All columns match: {all_columns_match}\n")
                    missing = set(all_schema_name) - set(all_cols)
                    extra = set(all_cols) - set(all_schema_name)
                    f.write(f"Missing columns: {missing}\n")
                    f.write(f"Extra columns: {extra}\n")
                # If datatypes do not match, write details about mismatched columns
                if not all_dtypes_match:
                    f.write(f"All dtypes match: {all_dtypes_match}\n")
                    for i, (expected, actual) in enumerate(zip(all_schema_dtype, all_dtypes)):
                        if expected != actual:
                            f.write(f"Column '{all_cols[i]}' dtype mismatch: expected {expected}, got {actual}\n")
            return validation_status
        
        except Exception as e:
            raise e

In [13]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValiadtion(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-06-08 23:40:41,123: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-06-08 23:40:41,124: INFO: common: yaml file: params.yaml loaded successfully]
[2025-06-08 23:40:41,125: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-06-08 23:40:41,126: INFO: common: created directory at: artifacts]
[2025-06-08 23:40:41,126: INFO: common: created directory at: artifacts/data_validation]
